In [1]:
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import openml
import openml
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *
from collections import Counter
from ucimlrepo import fetch_ucirepo


/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = openml.tasks.get_task(361260)
dataset = task.get_dataset()
X, a, b, c = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")

/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  exec(code_obj, self.user_global_ns, self.user_ns)
/scratch/users/zhongyuan_liang/conda/envs/mdi/lib/python3.10/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
Could not download file from https://openml1.win.tue

In [5]:
X.shape

(13932, 15)

In [5]:
c

['LATITUDE',
 'LONGITUDE',
 'LND_SQFOOT',
 'TOT_LVG_AREA',
 'SPEC_FEAT_VAL',
 'RAIL_DIST',
 'OCEAN_DIST',
 'WATER_DIST',
 'CNTR_DIST',
 'SUBCNTR_DI',
 'HWY_DIST',
 'age',
 'avno60plus',
 'month_sold',
 'structure_quality']

In [5]:
from sklearn import model_selection
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90  # X.shape[0] - X.shape[0] * 0.1
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None


    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [9]:
X = sample_real_data_X(source='openml', task_id=43, sample_row_n=150, seed=1)
print(np.sum(np.isnan(X)))
# y = linear_model_random_feature(X, beta=1, heritability=0.4, s=5, error_seed=1, feature_seed=1)

0


In [11]:
std

array([3.90111574e-01, 2.31691875e+00, 4.06950262e-01, 1.00000000e+00,
       5.88946936e-01, 1.95323595e-01, 3.85589309e-01, 3.98889926e-01,
       2.65639898e-01, 6.37253133e-01, 1.47377429e-01, 8.56447186e-01,
       1.98720172e-01, 1.99598486e-01, 3.81931122e-01, 7.26670058e-01,
       5.68560463e-01, 4.16324660e-01, 1.84963948e+00, 5.90150700e-01,
       1.25367383e+00, 8.42496723e-01, 3.79286465e-01, 1.93728837e-01,
       2.04827038e+00, 5.31901094e-01, 1.44054315e+00, 5.67991721e-01,
       8.38734719e-01, 2.65895961e-01, 1.75742501e-01, 1.64743788e-01,
       3.85275365e-01, 2.99715154e-01, 2.63724848e-01, 5.23057915e-01,
       5.06568412e-02, 4.30172286e-01, 2.73554577e-01, 3.79386294e-01,
       6.25106910e-01, 3.06825205e-01, 3.92975029e-01, 1.83274439e+00,
       4.65049527e-01, 1.42419974e-01, 2.02277796e-01, 6.09163175e-02,
       1.95280004e-01, 5.18212848e-02, 3.41071276e-01, 1.90183416e-01,
       9.09784895e-02, 2.53233198e+01, 1.24179933e+02, 3.82986062e+02])

In [7]:
X.mean(axis=0)

array([1.03600000e-01, 4.78533333e-01, 2.28533333e-01, 0.00000000e+00,
       2.97800000e-01, 7.44000000e-02, 1.39533333e-01, 1.20400000e-01,
       9.63333333e-02, 1.87333333e-01, 4.84000000e-02, 5.17133333e-01,
       6.36000000e-02, 3.46666667e-02, 4.88666667e-02, 2.41400000e-01,
       1.91000000e-01, 1.47333333e-01, 1.73266667e+00, 1.31466667e-01,
       8.17066667e-01, 8.29333333e-02, 7.66666667e-02, 5.22666667e-02,
       6.18666667e-01, 1.81600000e-01, 2.57666667e-01, 1.40266667e-01,
       1.26933333e-01, 6.99333333e-02, 4.18000000e-02, 3.14666667e-02,
       7.66000000e-02, 7.96000000e-02, 9.07333333e-02, 1.37466667e-01,
       7.86666667e-03, 9.89333333e-02, 6.84000000e-02, 5.02000000e-02,
       1.20133333e-01, 7.36000000e-02, 7.86000000e-02, 3.73466667e-01,
       1.08733333e-01, 1.54666667e-02, 2.94000000e-02, 2.23400000e-02,
       1.45653333e-01, 1.42333333e-02, 1.99773333e-01, 6.29600000e-02,
       1.99666667e-02, 5.63996000e+00, 4.45866667e+01, 2.36420000e+02])

In [8]:
X.std(axis=0)

array([3.90111574e-01, 2.31691875e+00, 4.06950262e-01, 0.00000000e+00,
       5.88946936e-01, 1.95323595e-01, 3.85589309e-01, 3.98889926e-01,
       2.65639898e-01, 6.37253133e-01, 1.47377429e-01, 8.56447186e-01,
       1.98720172e-01, 1.99598486e-01, 3.81931122e-01, 7.26670058e-01,
       5.68560463e-01, 4.16324660e-01, 1.84963948e+00, 5.90150700e-01,
       1.25367383e+00, 8.42496723e-01, 3.79286465e-01, 1.93728837e-01,
       2.04827038e+00, 5.31901094e-01, 1.44054315e+00, 5.67991721e-01,
       8.38734719e-01, 2.65895961e-01, 1.75742501e-01, 1.64743788e-01,
       3.85275365e-01, 2.99715154e-01, 2.63724848e-01, 5.23057915e-01,
       5.06568412e-02, 4.30172286e-01, 2.73554577e-01, 3.79386294e-01,
       6.25106910e-01, 3.06825205e-01, 3.92975029e-01, 1.83274439e+00,
       4.65049527e-01, 1.42419974e-01, 2.02277796e-01, 6.09163175e-02,
       1.95280004e-01, 5.18212848e-02, 3.41071276e-01, 1.90183416e-01,
       9.09784895e-02, 2.53233198e+01, 1.24179933e+02, 3.82986062e+02])

In [ ]:
(X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
# check if NA in X
print(np.sum(np.isnan(X)))

In [ ]:
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train, y_train)

rf_plus_baseline = RandomForestPlusRegressor(rf_model=est,
                                             prediction_model=LinearRegression(),
                                             fit_on="inbag",
                                             include_raw=False)
rf_plus_baseline.fit(X_train, y_train, n_jobs=None)

In [ ]:
tree = est.estimators_[57]
tree_rules = export_text(tree)
print(tree_rules)

In [ ]:
def LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    assert isinstance(fit, RandomForestPlusRegressor) or isinstance(fit, RandomForestPlusClassifier)
    rf_plus_mdi = RFPlusMDI(fit, mode = 'only_k', evaluate_on="all")
    local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
    local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

In [ ]:
lmdi_train, lmdi_test = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_baseline, mode="absolute")

In [ ]:
X = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361260.csv')
y = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361260.csv')

X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train, y_train)

rf_plus_baseline = RandomForestPlusRegressor(rf_model=est,
                                             prediction_model=LinearRegression(),
                                             fit_on="inbag",
                                             include_raw=False)
rf_plus_baseline.fit(X_train, y_train, n_jobs=None)

In [ ]:
def LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    assert isinstance(fit, RandomForestPlusRegressor) or isinstance(fit, RandomForestPlusClassifier)
    rf_plus_mdi = RFPlusMDI(fit, mode = 'only_k', evaluate_on="all")
    local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
    local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

lmdi_train, lmdi_test = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_baseline, mode="absolute")

In [ ]:
# np.max(np.abs(rf_plus_baseline.predict(X_train)-est.predict(X_train)))

In [ ]:
# X_train = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_X_train_16.npy")
# y_train = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_y_train_16.npy")
# weight = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_sample_weight_16.npy")
# # fit linear model
# # lr = LinearRegression()
# # lr.fit(X_train, y_train, sample_weight=weight)
# print(np.min(weight))

In [ ]:
X = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361254.csv')
y = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361254.csv')

X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=0)
est.fit(X_train, y_train)

rf_plus_baseline = RandomForestPlusRegressor(rf_model=est,
                                             prediction_model=LinearRegression(),
                                             fit_on="inbag",
                                             include_raw=False)
rf_plus_baseline.fit(X_train, y_train, n_jobs=None)

In [ ]:
X = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361254.csv')
y = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361254.csv')

X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=0)
est.fit(X_train, y_train)

rf_plus_baseline = RandomForestPlusRegressor(rf_model=est,
                                             prediction_model=LinearRegression(),
                                             fit_on="inbag",
                                             include_raw=False)
rf_plus_baseline.fit(X_train, y_train, n_jobs=None)
assert np.allclose(rf_plus_baseline.predict(X_train),est.predict(X_train))

In [ ]:
X = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361260.csv')
y = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361260.csv')

In [ ]:
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train, y_train)

rf_plus_inbag = RandomForestPlusRegressor(rf_model=est, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
rf_plus_inbag.fit(X_train, y_train, n_jobs=None)

In [ ]:
np.allclose(rf_plus_inbag.predict(X_test), est.predict(X_test))

In [ ]:
np.max(rf_plus_inbag.predict(X_test) - est.predict(X_test))

In [ ]:
np.min(rf_plus_inbag.predict(X_test) - est.predict(X_test))

In [ ]:
np.mean(rf_plus_inbag.predict(X_test) - est.predict(X_test))

In [ ]:
est.predict(X_test)

In [ ]:
X_train = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_X_train_2.npy")
y_train = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_y_train_2.npy")
weight = np.load("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/tree_sample_weight_2.npy")
# fit linear model
lr = LinearRegression()
lr.fit(X_train, y_train, sample_weight=weight)

In [ ]:
# fit linear model
lr = LinearRegression()
lr.fit(X_train, y_train, sample_weight=weight)

In [ ]:
np.min(weight)

In [ ]:
print("tree_X_train dtype:", X_train.dtype)
print("tree_y_train dtype:", y_train.dtype)
print("tree_sample_weight dtype:", weight.dtype)
print(X_train.shape)
print(y_train.shape)
print(weight.shape)

In [ ]:
weight

In [ ]:
# compute the smallest weight
min_weight = np.min(weight)
print("min weight:", min_weight)
max_weight = np.max(weight)
print("max weight:", max_weight)

In [ ]:
tree_16 = est.estimators_[16]
# generate inbad indices for tree 16
from sklearn.ensemble._forest import _generate_unsampled_indices, _generate_sample_indices
unsampled_indices = _generate_unsampled_indices(tree_16.random_state, X_train.shape[0], X_train.shape[0])
sample_indices = _generate_sample_indices(tree_16.random_state, X_train.shape[0], X_train.shape[0])

In [ ]:
sample_indices

In [ ]:
unique_inbag_indices, counts_elements = np.unique(sample_indices, return_counts=True)  

In [ ]:
np.min(counts_elements)

In [ ]:
weight

In [ ]:
# check if X_train is linearly independent
print(np.linalg.matrix_rank(X_train))

In [ ]:
np.max(np.abs(pd.DataFrame(X_train).corr() - np.eye(X_train.shape[1])))

In [ ]:
tree_41 = est.estimators_[16]
n_splits = tree_41.tree_.node_count - tree_41.tree_.n_leaves
print(f"The 41st tree has {n_splits} splits.")

In [ ]:
lr.predict(X_train)

In [ ]:
np.max(np.abs(pd.DataFrame(X_train).corr() - np.eye(X_train.shape[1])))

In [ ]:
X_train.shape

In [ ]:
# get diabetes_regr from imodels
from sklearn.model_selection import train_test_split
from imodels import get_clean_dataset, HSTreeClassifierCV # import any imodels model here

# prepare data (a sample clinical dataset)
X, y, feature_names = get_clean_dataset('diabetes_regr')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import RidgeCV
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)

rf_plus_ridge = RandomForestPlusRegressor(rf_model=est, prediction_model=RidgeCV(cv=5))
rf_plus_ridge.fit(X_train, y_train)

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_ridge, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
local_fi_score_train_bootstrap = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, bootstrap=50000)

In [ ]:
np.sum(np.argsort(local_fi_score_train_bootstrap) == np.argsort(local_fi_score_train))

In [ ]:
local_fi_score_train_bootstrap.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Seed for reproducibility
np.random.seed(42)

# Generate a random sample of 100 numbers from a normal distribution
original_sample = np.random.normal(loc=50, scale=10, size=100)

# Bootstrap settings
n_bootstraps = 10000
bootstrap_means = np.empty(n_bootstraps)

# Perform bootstrap resampling
for i in range(n_bootstraps):
    bootstrap_sample = np.random.choice(original_sample, size=original_sample.size, replace=True)
    bootstrap_means[i] = np.mean(bootstrap_sample)

# Original sample mean
original_sample_mean = np.mean(original_sample)

# Mean of bootstrap means
mean_of_bootstrap_means = np.mean(bootstrap_means)

# Plotting
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_means, bins=30, alpha=0.7, color='blue', label='Bootstrap Means')
plt.axvline(x=original_sample_mean, color='red', linestyle='--', label='Original Sample Mean')
plt.axvline(x=mean_of_bootstrap_means, color='green', linestyle='-', label='Mean of Bootstrap Means')
plt.title('Distribution of Bootstrap Means')
plt.xlabel('Means')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Output the means for comparison
original_sample_mean, mean_of_bootstrap_means


In [ ]:
np.argsort(local_fi_score_train)

In [ ]:
np.argsort(local_fi_score_train_bootstrap)

In [ ]:
task_id = 361242
task = openml.tasks.get_task(task_id)
dataset = task.get_dataset()
X, _, _, _ = dataset.get_data(target=dataset.default_target_attribute,dataset_format="array")


In [ ]:
np.random.seed(4307)
keep_idx = np.random.choice(X.shape[0], 2000, replace=False)
X = X[keep_idx, :]
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
X = sample_real_data_X(source="openml", task_id = 361242, normalize=True)

#sample_real_data_y(source="openml", task_id = 361243)[0] #361243

In [ ]:
y = linear_model_random_feature(X, beta=1, heritability=0.2, s=5, error_seed=3, feature_seed=3)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LassoCV
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)

rf_plus_lasso = RandomForestPlusRegressor(rf_model=est, prediction_model=LassoCV(cv=5, max_iter=10000, random_state=0))
rf_plus_lasso.fit(X_train, y_train)

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_lasso, mode = 'only_k', evaluate_on="all")
local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)

In [ ]:
score1.shape

In [ ]:
score1_rank.shape

In [ ]:
#plot np.nanmean(score1_rank[:,2,:],axis=-1)
plt.plot(np.nanmean(score1_rank[:,2,:],axis=-1))
plt.show()

In [ ]:
score1_rank[:,2,:].ravel()

In [ ]:
#plot np.nanmean(score1_rank[:,2,:],axis=-1)
plt.plot(score1_rank[:,2,:])
plt.show()

In [ ]:
np.argsort(-1*np.mean(np.nanmean(score1,axis=-1), axis=0))

In [ ]:
np.argsort(-1*np.mean(np.nanmean(score1_rank,axis=-1), axis=0))

In [ ]:
def tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    """
    Compute average treeshap value across observations.
    Larger absolute values indicate more important features.
    :param X: design matrix
    :param y: response
    :param fit: fitted model of interest (tree-based)
    :return: dataframe of shape: (n_samples, n_features)
    """
    explainer = shap.TreeExplainer(fit)
    local_fi_score_train = explainer.shap_values(X_train, check_additivity=False)
    local_fi_score_test = explainer.shap_values(X_test, check_additivity=False)
    if sklearn.base.is_classifier(fit):
        if mode == "absolute":
            return np.abs(local_fi_score_train[:,:,1]), np.abs(local_fi_score_test[:,:,1])
        else:
            return local_fi_score_train[:,:,1], local_fi_score_test[:,:,1]
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

score2, _ = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=est, mode="absolute")

In [ ]:
np.argsort(-1*np.mean(score1, axis=0))

In [ ]:
np.argsort(-1*np.mean(score2[0], axis=0))

In [ ]:
# Fit a random forest model on features 17, 2, 16
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train[:,[17,2,16]], y_train)

#evalaute the model
y_pred = est.predict(X_test[:,[17,2,16]])
r2_score(y_test, y_pred)

In [ ]:
# Fit a linear model on X_train and print coefficients
est = RidgeCV(cv=5)
est.fit(X_train, y_train)
est.coef_

In [ ]:
# print correlation of X_train as dataframe
pd.set_option('display.max_columns', None)
pd.DataFrame(X_train).corr()

In [ ]:
# plot score1[:, 2]
plt.plot(score1[:, 2])
plt.xlabel("Sample index")
plt.ylabel("Feature importance")

In [ ]:
plt.plot(score1[:, 20])
plt.xlabel("Sample index")
plt.ylabel("Feature importance")

In [ ]:
np.mean(score1, axis=0)

In [ ]:
# Fit a random forest model on features 17, 2, 16
est = RandomForestRegressor(random_state=1)
est.fit(X_train[:,[2,16,17]], y_train)

#evalaute the model
y_pred = est.predict(X_test[:,[2,16,17]])
r2_score(y_test, y_pred)

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_base, mode = 'only_k', evaluate_on="oob")
score4 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking=True)

In [ ]:
score3

In [ ]:
score4

In [ ]:
sum = 0
for i in range(X_train.shape[0]):
    sort_coef = np.argsort(-1 * np.argsort(np.argsort(np.abs(score3[i]))))
    sort_loo_coefficients = np.argsort(-1 * np.argsort(np.argsort(np.abs(score4[i]))))
    if np.array_equal(sort_coef[:5], sort_loo_coefficients[:5]):
        sum += 1
print(sum)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, bootstrap=True)

In [ ]:
rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)

In [ ]:
np.argsort(rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, bootstrap=True))

In [ ]:
np.argsort(rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train))

In [ ]:
# CHECK how many features are used in the random forest
# Extract the unique features used in the tree
tree = est.estimators_[0]
features_used = set(tree.tree_.feature[tree.tree_.feature != -2])
print("Indices of features used in the tree:", features_used)
print("Number of unique features used in the tree:", len(features_used))

In [ ]:
explainer = shap.TreeExplainer(est)
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag", mode="only_k")
shap_lfi = explainer.shap_values(X_train, check_additivity=False)
lmdi_lfi= rf_plus_mdi.explain_linear_partial(X=X_train, y=None)

#### Single Data Point

In [ ]:
sample_index = 0
sample = X_train[sample_index].reshape(1, -1)

# Get the single tree
tree = est.estimators_[0]

# Find features used in the decision path
used_features = set()
node_indicator = tree.decision_path(sample)
feature_index = tree.tree_.feature

# Collect features used in the path for the sample
path_nodes = node_indicator.indices
for node in path_nodes:
    if feature_index[node] != -2:  # Ignore leaf nodes
        used_features.add(feature_index[node])

# Print the feature indices used in the decision path
print("Features used in the path for X_train[0]:", sorted(used_features))
print("Feature names used:", [f"feature_{i}" for i in sorted(used_features)])

In [ ]:
lmdi_lfi[0]

In [ ]:
shap_lfi[0]

#### Verify whether positive beta*x leads to positive y_hat changes

In [ ]:
neg_shap_rank = np.argsort(shap_lfi)
neg_lmdi_rank = np.argsort(lmdi_lfi)
pos_shap_rank = np.argsort(-shap_lfi)
pos_lmdi_rank = np.argsort(-lmdi_lfi)

In [ ]:
train_mean = np.mean(X_train, axis=0)
#train_mean = np.zeros(X_train.shape[1])

In [ ]:
y_pred_train = est.predict(X_train)

In [ ]:
data_copy = X_train.copy()
indices = neg_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_neg = est.predict(data_copy)
array = y_pred_train_shap_neg-y_pred_train
print("sum:", sum)
np.sum(array != 0)

In [ ]:
data_copy = X_train.copy()
indices = pos_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_shap_pos
print("sum:", sum)
np.sum(array != 0)

In [ ]:
data_copy = X_train.copy()
indices = neg_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_lmdi_neg = est.predict(data_copy)
array = y_pred_train_lmdi_neg-y_pred_train
print(f"Sum: {sum}")
np.sum(array != 0)

In [ ]:
data_copy = X_train.copy()
indices = pos_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
    else:
        print(i)
y_pred_train_lmdi_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_lmdi_pos
print(f"Sum: {sum}")
np.sum(array != 0)

In [ ]:
data_copy = X_train.copy()
indices = pos_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_shap_pos
print("sum:", sum)
np.sum(array > 0)

In [ ]:
data_copy = X_train.copy()
indices = neg_shap_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if shap_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_shap_neg = est.predict(data_copy)
array = y_pred_train_shap_neg-y_pred_train
print("sum:", sum)
np.sum(array > 0)

In [ ]:
data_copy = X_train.copy()
indices = pos_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] > 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
    else:
        print(i)
y_pred_train_lmdi_pos = est.predict(data_copy)
array = y_pred_train-y_pred_train_lmdi_pos
print(f"Sum: {sum}")
np.sum(array > 0)

In [ ]:
data_copy = X_train.copy()
indices = neg_lmdi_rank[:, 0]
sum = 0
for i in range(X_train.shape[0]):
    if lmdi_lfi[i, indices[i]] < 0:
        if lmdi_lfi[i, indices[i]] < 0:
            data_copy[i, indices[i]] = 9999999
        else:
            data_copy[i, indices[i]] = -1*9999999
        sum += 1
y_pred_train_lmdi_neg = est.predict(data_copy)
array = y_pred_train_lmdi_neg-y_pred_train
print(f"Sum: {sum}")
np.sum(array > 0)

In [ ]:
lmdi_lfi[9]

In [ ]:
lmdi_lfi[0]

In [ ]:
array

In [ ]:
X_train.shape

In [ ]:
rf_plus_mdi = RFPlusMDI(rf_plus_base_inbag, evaluate_on="inbag", mode="only_k")

In [ ]:
est.predict(X_train[:10])

In [ ]:
rf_plus_base_inbag.predict(X_train[:10])

In [ ]:
lfi1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi1

In [ ]:
lfi1 = rf_plus_mdi.explain_linear_partial(X=X_train, y=None)
lfi1

In [ ]:
import numpy as np

# Check how many rows in lfi1 have NaN values
nan_rows = lfi1[np.isnan(lfi1).any(axis=1)]


In [ ]:
nan_rows

In [ ]:
lfi1[0]

In [ ]:
explainer = shap.TreeExplainer(est)
explainer.shap_values(X_train, check_additivity=False)[0]

In [ ]:
explainer.shap_values(X_train, check_additivity=False)[0]

In [ ]:
temp = X_train[0]
temp

In [ ]:
temp[5] = 100

In [ ]:
est.predict(np.array([temp]))

In [ ]:
explainer.expected_value

In [ ]:
alo_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all", mode="only_k")

In [ ]:
lfi2 = alo_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi2

In [ ]:
lfi1[0].sum()

In [ ]:
lfi2 = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train)
lfi2

In [ ]:
# np.random.seed(42) 
# data = np.random.randn(1000, 10)
# n_groups = 2
# group_indicator = np.random.choice(n_groups, size=1000)
# y = np.zeros(1000)
# coefficients = np.random.randn(n_groups, data.shape[1])
# for group in range(n_groups):
#     group_mask = group_indicator == group
#     selected_features = data[group_mask]
#     y[group_mask] = np.dot(selected_features, coefficients[group])
# X = np.column_stack((data, group_indicator))

In [ ]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
pd.DataFrame(X).corr()

In [ ]:
# Plot y
plt.hist(y, bins=50)

In [ ]:
# apply the log transformation to y
# y = np.log(y)
# plt.hist(y, bins=50)

In [ ]:
# X, y, _ = imodels.get_clean_dataset("diabetes_regr")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape)
# standardize the data using sklearn's StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
# est.fit(X_train, y_train)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est)
# rf_plus_base.fit(X_train, y_train)

In [ ]:
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)

In [ ]:
import lime
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *

In [ ]:
a,b,c,d = LFI_evaluation_RFPlus_oob(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=rf_plus_base, mode="absolute", train_only=False)

In [ ]:
lime_evaluation_RF(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=est, mode="absolute", train_only=True)

In [ ]:
lime = lime_evaluation_RF(X_train, y_train, None, None, X_test, y_test, None, None, fit=est, mode="absolute", train_only=True)[0]

In [ ]:
treeshap = tree_shap_evaluation_RF(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=est, mode="absolute", train_only=True)[0]

In [ ]:
lmdi = LFI_evaluation_RFPlus_all(X_train, y_train, X_train, y_train, X_test, y_test, X_test, y_test, fit=rf_plus_base, mode="absolute", train_only=True)[0]

In [ ]:
treeshap.shape

In [ ]:
column_means = np.mean(treeshap, axis=0)
sorted = np.argsort(-column_means)
sorted

In [ ]:
def select_top_features(array, sorted_indices, percentage):
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * percentage))
    selected_indices = sorted_indices[:num_selected]
    selected_array = array[:, selected_indices]
    return num_selected, selected_array

In [ ]:
X_train[0]

In [ ]:
X_test[0]

In [ ]:
select_top_features(X_test, sorted, 0.25)[0]

In [ ]:
select_top_features(X_train, sorted, 0.25)[0]

In [ ]:
column_means = np.mean(lime, axis=0)
np.argsort(-column_means)

In [ ]:
column_means = np.mean(lmdi, axis=0)
np.argsort(-column_means)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode="regression")
result = np.zeros((X_train.shape[0], X_train.shape[1]))
for i in range(X_train.shape[0]):
    exp = explainer.explain_instance(X_train[i,:], est.predict, num_features=X_train.shape[1])
    original_feature_importance = exp.as_map()[1]
    sorted_feature_importance = sorted(original_feature_importance,key = lambda x: x[0])
    for j in range(X_train.shape[1]):
        result[i,j] = sorted_feature_importance[j][1] #abs(sorted_feature_importance[j][1])

# Convert the array to a DataFrame
lime_values = pd.DataFrame(result, columns=[f'Feature_{i}' for i in range(X_train.shape[1])])
lime_values = lime_values

In [ ]:
lime_values

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode="regression")
result = np.zeros((X_train.shape[0], X_train.shape[1]))
i = 0
exp = explainer.explain_instance(X_train[i,:], est.predict, num_features=X_train.shape[1])

In [ ]:
exp.as_list()

In [ ]:
exp.as_map()[1]

In [ ]:
sorted_feature_importance = sorted(exp.as_map()[1],key = lambda x: x[0])
sorted_feature_importance

In [ ]:
sorted_feature_importance[0][1]

In [ ]:
def explain(self, X_train,X_test,num_features = 10): #For experiments change based on number of features we are ablating 
        
        # get shape of X_test
        if X_test is None: #assume we are explaining training set
            X_to_explain = copy.deepcopy(X_train)   
            n_samples, num_features = X_train.shape
        else: #assume we are explaining test set
            X_to_explain = copy.deepcopy(X_test)
            n_samples, num_features = X_test.shape
        
        # create data structure to save scores in
        result = np.zeros((n_samples, num_features))
        
        # initialize the LIME explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(X_train,verbose=False,mode=self.task)
        
        for i in range(n_samples):
            exp = explainer.explain_instance(X_to_explain[i,:], self.model_pred_func,num_features=num_features)
            original_feature_importance = exp.as_map()[1]
            sorted_feature_importance = sorted(original_feature_importance,key = lambda x: x[0])
            for j in range(num_features):
                result[i,j] = sorted_feature_importance[j][1] #abs(sorted_feature_importance[j][1])
        
        # Convert the array to a DataFrame
        lime_values = pd.DataFrame(result, columns=[f'Feature_{i}' for i in range(num_features)])
        lime_values = lime_values #abs(lime_values)
        
        return lime_values

In [ ]:
from sklearn.kernel_ridge import KernelRidge
est = KernelRidge()
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': [0.1, 0.01, 0.001, None]
}
grid_search = GridSearchCV(KernelRidge(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_est = grid_search.best_estimator_
y_pred = best_est.predict(X_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

In [ ]:
grid_search.best_params_

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_leaf= 3, max_features= 'sqrt', random_state= 42)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusClassifier(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

In [ ]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
partial_preds_subtract_intercept = rf_plus_mdi.explain_subtract_intercept(X=X_test)

In [ ]:
partial_preds_subtract_intercept[0]

In [ ]:
partial_preds_subtract_intercept[0]

In [ ]:
rf_plus_base.predict_proba(X_test)

In [ ]:
temp = rf_plus_mdi.explain(X=X_test, y=y_test)

In [ ]:
temp[0][0]

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=41, min_samples_leaf=5, max_features=0.33)
rf.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf))
rf_plus_base.fit(X_train, y_train)
test_all_mse_rf = mean_squared_error(y_test, rf.predict(X_test))
test_all_r2_rf = r2_score(y_test, rf.predict(X_test))
test_all_mse_rf_plus = mean_squared_error(y_test, rf_plus_base.predict(X_test))
test_all_r2_rf_plus = r2_score(y_test, rf_plus_base.predict(X_test))
print("Test MSE RF: ", test_all_mse_rf)
print("Test R2 RF: ", test_all_r2_rf)
print("Test MSE RF+: ", test_all_mse_rf_plus)
print("Test R2 RF+: ", test_all_r2_rf_plus)

In [ ]:
# Get shap values
import shap
explainer = shap.TreeExplainer(rf)
shap_values_train = explainer.shap_values(X_train, check_additivity=True)
# shap_values_train = np.abs(shap_values_train)
shap_values_test = explainer.shap_values(X_test, check_additivity=True)
# shap_values_test = np.abs(shap_values_test)

In [ ]:
def ablation_removal(train_mean, data, feature_importance_rank, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    data_copy[np.arange(data.shape[0]), indices] = train_mean[indices]
    return data_copy
def ablation_removal_positive(train_mean, data, feature_importance_rank, feature_importance, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    sum = 0
    for i in range(data.shape[0]):
        if feature_importance[i, indices[i]] > 0:
            sum += 1
            data_copy[i, indices[i]] = train_mean[indices[i]]
    print("Remove sum: ", sum)
    return data_copy
def ablation_removal_negative(train_mean, data, feature_importance_rank, feature_importance, feature_index):
    data_copy = data.copy()
    indices = feature_importance_rank[:, feature_index]
    sum = 0
    for i in range(data.shape[0]):
        if feature_importance[i, indices[i]] < 0:
            sum += 1
            data_copy[i, indices[i]] = train_mean[indices[i]]
    print("Remove sum: ", sum)
    return data_copy

In [ ]:
ablation_models = {"RF_Regressor": rf,
                    "Linear": LinearRegression(),
                    "RF_Plus_Regressor": rf_plus_base}
X_data = X_test
y_data = y_test
ablation_data="test"
ablation_models["Linear"].fit(X_train, y_train)

### Shap

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    print(f"enter i: {i}")
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data =shap_values_test
local_fi_score_data_rank = np.argsort(-1*shap_values_test)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "RF_Plus_Regressor"
ablation_est = ablation_models[a_model]
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(shap_values_test))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

### Check whether sum to hat_y

In [ ]:
assert np.all(rf.predict(X_test)) == np.all(np.sum(shap_values_test, axis=1) + explainer.expected_value)

In [ ]:
explainer.expected_value

In [ ]:
# get localMDI+
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_feature_importances_train, a = rf_plus_mdi.explain(X=X_train, y=y_train)
local_feature_importances_train[0]

In [ ]:
from sklearn.linear_model import Ridge
rf_plus_base = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf), include_raw=False, fit_on="inbag", prediction_model=Ridge(alpha=1e-6))
rf_plus_base.fit(X_train, y_train)
rf_plus_mdi = RFPlusMDI(rf_plus_base, evaluate_on="inbag")
local_feature_importances_train, _ = rf_plus_mdi.explain(X=X_train, y=y_train)
local_feature_importances_train[0]

### LMDI+

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data =local_feature_importances_train
local_fi_score_data_rank = np.argsort(local_feature_importances_train)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_negative(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data =local_feature_importances_train
local_fi_score_data_rank = np.argsort(-1*local_feature_importances_train)
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, local_fi_score_data_rank, local_fi_score_data, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
a_model = "Linear"
ablation_est = ablation_models[a_model]
X_data = X_train
y_data = y_train
ablation_data="test"
metric_results = {}
local_fi_score_data_rank = np.argsort(-1*np.abs(local_feature_importances_train))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))
num_ablate_features = X_train.shape[1]
train_mean = np.mean(X_train, axis=0)

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal(train_mean, X_temp, local_fi_score_data_rank, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

mse_before = metric_results[a_model + f'_{ablation_data}_MSE_before_ablation']
r2_before = metric_results[a_model + f'_{ablation_data}_R_2_before_ablation']

mse_after = [metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i}'] for i in range(1, num_ablate_features)]
r2_after = [metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i}'] for i in range(1, num_ablate_features)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

#### Analysis 1

In [ ]:
pd.DataFrame(X_train).corr()

In [ ]:
local_fi_score_data = np.argsort(-1*local_feature_importances_train)#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
local_fi_score_data = np.argsort(-1*np.random.rand(X_train.shape[0], 10))#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
mse_before = metric_results['Linear_train_MSE_before_ablation']
r2_before = metric_results['Linear_train_R_2_before_ablation']

mse_after = [metric_results[f'Linear_train_MSE_after_ablation_{i}'] for i in range(1, 10)]
r2_after = [metric_results[f'Linear_train_R_2_after_ablation_{i}'] for i in range(1, 10)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
local_fi_score_data = np.argsort(-1*shap_values_train)#np.argsort(-1*local_feature_importances_train) #np.argsort(-1*shap_values_train) #np.argsort(-1*np.random.rand(X_train.shape[0], 10))

y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    print(differences_per_col)
    lm = LinearRegression()
    lm.fit(X_temp, y_train)
    print(r2_score(y_train, lm.predict(X_temp)))
for i in range(num_ablate_features):
    metric_results[f'{a_model}_{ablation_data}_MSE_after_ablation_{i+1}'] = ablation_results_list[i]
    metric_results[f'{a_model}_{ablation_data}_R_2_after_ablation_{i+1}'] = ablation_results_list_r2[i]

In [ ]:
mse_before = metric_results['Linear_train_MSE_before_ablation']
r2_before = metric_results['Linear_train_R_2_before_ablation']

mse_after = [metric_results[f'Linear_train_MSE_after_ablation_{i}'] for i in range(1, 10)]
r2_after = [metric_results[f'Linear_train_R_2_after_ablation_{i}'] for i in range(1, 10)]

# Plotting MSE
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, 10)], [mse_before] + mse_after, color='tab:red', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('MSE')
plt.title(f'{a_model} MSE Before and After Ablation')
plt.show()

# Plotting R^2
plt.figure(figsize=(12, 6))
plt.plot(['Before'] + [f'After {i}' for i in range(1, num_ablate_features)], [r2_before] + r2_after, color='tab:blue', marker='o')
plt.xlabel('Ablation Step')
plt.ylabel('R^2')
plt.title(f'{a_model} R^2 Before and After Ablation')
plt.show()

In [ ]:
y_pred = ablation_est.predict(X_data)
metric_results[a_model + f'_{ablation_data}_MSE_before_ablation'] = mean_squared_error(y_data, y_pred)
metric_results[a_model + f'_{ablation_data}_R_2_before_ablation'] = r2_score(y_data, y_pred)
imp_vals = copy.deepcopy(local_fi_score_data)
ablation_results_list = [0] * num_ablate_features
ablation_results_list_r2 = [0] * num_ablate_features
X_temp = X_data.copy()
var = []
for i in range(num_ablate_features):
    ablation_X_data = ablation_removal_positive(train_mean, X_temp, imp_vals, i)
    ablation_results_list[i] = mean_squared_error(y_data, ablation_est.predict(ablation_X_data))
    ablation_results_list_r2[i] = r2_score(y_data, ablation_est.predict(ablation_X_data))
    X_temp = ablation_X_data
    differences = X_temp != X_train
    differences_per_col = np.sum(differences, axis=0)
    # print(differences_per_col)
    # print(np.array(differences_per_col).var())
    var.append(np.array(differences_per_col).var())

In [ ]:
ablation_results_list

In [ ]:
ablation_results_list_r2

In [ ]:
differences = X_temp != X_train
differences_per_row = np.sum(differences, axis=1)
print(differences_per_row)

In [ ]:
X_train.shape

In [ ]:
differences = X_temp != X_train
differences_per_col = np.sum(differences, axis=0)
print(differences_per_col)
print(np.array(differences_per_col).var())

In [ ]:
train_mean

In [ ]:
ablation_est.coef_

In [ ]:
ablation_est.intercept_

In [ ]:
y_train.mean()

In [ ]:
((y_train - y_train.mean())**2).sum()

In [ ]:
((y_train - ablation_est.predict(X_train))**2).sum()

In [ ]:
((y_train - ablation_est.predict(X_temp))**2).sum()

### Analysis 2

In [ ]:
lm = LinearRegression()
lm.fit(X_temp, y_train)
r2_score(y_train, lm.predict(X_temp))

In [ ]:
lm.coef_

In [ ]:
ablation_est.coef_

In [ ]:
lm.coef_-ablation_est.coef_